# Spotify Song Suggestor
-Natural Language Processing Model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from re import compile as rcompile
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors

In [2]:
from google.colab import files
file = files.upload()

Saving spotify_songs.csv to spotify_songs.csv


In [3]:
#dataset used https://www.kaggle.com/imuhammad/audio-features-and-lyrics-of-spotify-songs
df = pd.read_csv('spotify_songs.csv')

In [4]:
df.shape

(18454, 25)

In [5]:
df.isnull().sum()

track_id                      0
track_name                    0
track_artist                  0
lyrics                      260
track_popularity              0
track_album_id                0
track_album_name              0
track_album_release_date      0
playlist_name                 0
playlist_id                   0
playlist_genre                0
playlist_subgenre             0
danceability                  0
energy                        0
key                           0
loudness                      0
mode                          0
speechiness                   0
acousticness                  0
instrumentalness              0
liveness                      0
valence                       0
tempo                         0
duration_ms                   0
language                    260
dtype: int64

In [6]:
df.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,rock,classic rock,0.682,0.401,2,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,rock,hard rock,0.303,0.880,9,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,r&b,new jack swing,0.845,0.652,6,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,r&b,neo soul,0.425,0.378,5,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,pop,dance pop,0.760,0.887,9,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [7]:
def wrangle(data):
    #drop null values
    data = data.dropna()
    
    #restrict to songs with lyrics in English
    data = data[data['language']=='en']
    
    #drop duplicates
    data = data.drop_duplicates(subset = ['track_name', 'track_artist'],
                                             keep = 'first')
    
    #reset index
    data = data.reset_index()
    
    return data

In [8]:
df2 = wrangle(df)

In [9]:
df2.shape

(13726, 26)

In [10]:
df2.isnull().sum()

index                       0
track_id                    0
track_name                  0
track_artist                0
lyrics                      0
track_popularity            0
track_album_id              0
track_album_name            0
track_album_release_date    0
playlist_name               0
playlist_id                 0
playlist_genre              0
playlist_subgenre           0
danceability                0
energy                      0
key                         0
loudness                    0
mode                        0
speechiness                 0
acousticness                0
instrumentalness            0
liveness                    0
valence                     0
tempo                       0
duration_ms                 0
language                    0
dtype: int64

In [11]:
def gather_data(songs):
    data =[]
    for song in songs:
        data.append(df2['track_name'][df2['track_id']==song])
    
    new_data = []

    for song in data:
        str_song = pd.Series(song).item()
        new_data.append(str_song)
        
    return new_data

In [12]:
songs = df2['track_id']

data = gather_data(songs)

In [13]:
type(data)

list

In [14]:
len(data)

13726

In [53]:
display(df2.lyrics)

0        The trees, are singing in the wind The sky blu...
1        NA Yeah, Spyderman and Freeze in full effect U...
2        I really can't stay Baby it's cold outside I'v...
3        Get up out of my business You don't keep me fr...
4        Hold your breath, don't look down, keep trying...
                               ...                        
13721    I had to catch the bus Alarm was shaking, oh A...
13722    Yeah, nah, nah, nah, nah, nah, nah, nah, nah N...
13723    Caught up in such a head rush, wide-eyed latel...
13724    Bound, bound Bound, bound Bound to fall in lov...
13725    (Would you do for me) Sweetheart (Would you do...
Name: lyrics, Length: 13726, dtype: object

In [15]:
df2.head()

,index,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,rock,hard rock,0.303,0.880,9,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
1,2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,r&b,new jack swing,0.845,0.652,6,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
2,3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,r&b,neo soul,0.425,0.378,5,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
3,4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,pop,dance pop,0.760,0.887,9,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en
4,5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying...",70,3GNzXsFbzdwM0WKCZtgeNP,Chapters,2019-04-26,urban contemporary,4WiB26kw0INKwbzfb5M6Tv,r&b,urban contemporary,0.496,0.639,6,-6.157,1,0.0550,0.28000,0.00000,0.0975,0.305,147.764,224720,en


In [16]:
vect = TfidfVectorizer(stop_words='english', 
                        ngram_range=(1,2), min_df=0.03, 
                        max_df=0.25) 

# Create a vocabulary and get word counts per document 
dtm = vect.fit_transform(data) 

# Get feature names to use as dataframe column headers 
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names()) 
print(dtm.shape) 
dtm.head()

(13726, 3)


,feat,love,remix
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [17]:
dtm_filename = 'nlp_dtm.pkl' 
pickle.dump(dtm, open(dtm_filename, 'wb'))

In [18]:
dtm_filename = 'nlp_dtm.pkl'

dtm = pickle.load(open(dtm_filename, 'rb'))

In [19]:
nn = NearestNeighbors(n_neighbors=6, algorithm='kd_tree') 
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

In [20]:
model_filename = 'nlp_model.pkl' 
pickle.dump(nn, open(model_filename, 'wb'))

In [21]:
dtm.tail()

,feat,love,remix
13721,0.0,0.0,0.0
13722,0.0,0.0,0.0
13723,0.0,0.0,0.0
13724,0.0,0.0,0.0
13725,0.0,0.0,0.0


In [22]:
df2.tail()

,index,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
13721,18447,7zxLkZbUxITHabPzGN8Xgc,Sick Feeling,boy pablo,"I had to catch the bus Alarm was shaking, oh A...",64,1lESM5Y1VAvrCLEgLmyfOw,Soy Pablo,2018-10-05,②⓪①⑨ mixed,2bOjjgN1S3Gqd8vSMyafvJ,rock,permanent wave,0.425,0.504,9,-10.003,1,0.0314,0.02200,0.000004,0.3630,0.403,165.818,155714,en
13722,18448,7zxRMhXxJMQCeDDg0rKAVo,Some Way,NAV,"Yeah, nah, nah, nah, nah, nah, nah, nah, nah N...",72,4JR29CNW14Zc4Z5vflxkoJ,NAV,2017-02-24,PROJECT: Contemporary,6HaCi9bqaiuSZEDfCEmwyo,r&b,urban contemporary,0.744,0.715,0,-6.103,1,0.3510,0.10100,0.000000,0.0919,0.340,154.962,179773,en
13723,18449,7zXzWCVuz7whIuYZyEAlxt,Rising Like The Sun - Radio Mix,Qulinez,"Caught up in such a head rush, wide-eyed latel...",0,1l4aoukbPgi5u2OaE2R4Zj,Rising Like The Sun,2014-03-24,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.479,0.800,10,-5.778,0,0.0878,0.00555,0.000000,0.3350,0.211,128.012,208656,en
13724,18451,7zye9v6B785eFWEFYs13C2,Bound,Ponderosa Twins Plus One,"Bound, bound Bound, bound Bound to fall in lov...",40,1xdgLmTFMSyJyI5DJOOX7T,2+2+1 = (Digitally Remastered),2013-07-09,Sexy Soul 2020,5EMARioe9z9eKOeWIAC2JW,r&b,neo soul,0.458,0.540,5,-6.457,0,0.0270,0.71500,0.000428,0.1150,0.657,142.218,191205,en
13725,18452,7zyLObYw4QUKQDyZOb4J0Y,I'll Do 4 U (Re-Recorded / Remastered),Father MC,(Would you do for me) Sweetheart (Would you do...,36,14HYMxFhpgDIr9cci1u0kt,I'll Do 4 U (Re-Recorded / Remastered),2010-10-01,New Jack Swing/ R&B Hits: 1987 - 2002,4sji14lrB5bgcr51lPALYH,r&b,new jack swing,0.832,0.666,1,-4.920,0,0.0633,0.14300,0.000000,0.0720,0.810,109.536,223890,en


In [28]:
def select_nearest_songs(id):
    
    track_num = df2.loc[df2['track_id']==id]
    x = track_num.index
    x = x[0]
    x = x.item()

    doc = dtm.loc[x].values
    result = nn.kneighbors([doc], n_neighbors=6)
    
    rec_songs = {"id":[]};
    
    for i in range(5):
        song = result[1][0][1+i]

        #translate loc
        id = df.loc[song]['track_id']

        
        rec_songs['id'].append(id)
    
    return rec_songs

In [29]:
select_nearest_songs('00cqd6ZsSkLZqGMlQCR0Zo')

{'id': ['00chLpzhgVjxs1zKC9UScL',
  '3GE52DOmiSQ2R3FHGmYSif',
  '2BWXfyYzeLcK0pbvPp9Itw',
  '3gEaSrF7vC2FhfAARGeCoK',
  '0yndiszlOMgxOu3y2vKZvB']}

In [30]:
select_nearest_songs('004s3t0ONYlzxII9PLgU6z')

{'id': ['1oUpRnKn36YRqAyCAxHOqT',
  '0C7dYeCLF6ukChtW64UNge',
  '03jdl5GTBhKsgB1Yw8mVPh',
  '07e2UjDMuFZyOiu8Xo3CwX',
  '0WNGPpmWqzPnk0psUhJ3SX']}

In [31]:
select_nearest_songs('7zye9v6B785eFWEFYs13C2')

{'id': ['1oUpRnKn36YRqAyCAxHOqT',
  '0C7dYeCLF6ukChtW64UNge',
  '03jdl5GTBhKsgB1Yw8mVPh',
  '07e2UjDMuFZyOiu8Xo3CwX',
  '0WNGPpmWqzPnk0psUhJ3SX']}

In [32]:
select_nearest_songs('7zyLObYw4QUKQDyZOb4J0Y')

{'id': ['1oUpRnKn36YRqAyCAxHOqT',
  '0C7dYeCLF6ukChtW64UNge',
  '03jdl5GTBhKsgB1Yw8mVPh',
  '07e2UjDMuFZyOiu8Xo3CwX',
  '0WNGPpmWqzPnk0psUhJ3SX']}

In [ ]:
print(f'numpy=={np.__version__}')
print(f'pandas=={pd.__version__}')
#print(f'scikit-learn=={sklearn.__version__}')

numpy==1.19.5
pandas==1.1.5
